In [1]:
import os 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import matplotlib as mpl 
# import missingno as msno
import seaborn as sns

from src import config

In [2]:
df = pd.read_csv(config.RAW_FILE_PATH / 'olist_order_items_dataset.csv')

print(f'''
{df.info()}
{print(df.isna().sum())}
{df.shape}
{df.head()}
''')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB
order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

None
None
(112650, 7)
                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1

In [3]:
df = df.groupby('order_id').agg({'product_id': lambda x: list(x)})

In [4]:
!pip install pyspark
!pip install findspark

     |████████████████████████████████| 281.4 MB 122 kB/s eta 0:00:011   |█▏                              | 10.3 MB 3.8 MB/s eta 0:01:12     |█████████▏                      | 80.4 MB 231 kB/s eta 0:14:27     |███████████▍                    | 100.3 MB 9.4 MB/s eta 0:00:20     |████████████████▎               | 143.4 MB 15.6 MB/s eta 0:00:09     |████████████████▊               | 146.7 MB 15.6 MB/s eta 0:00:09     |█████████████████               | 149.2 MB 15.3 MB/s eta 0:00:09     |█████████████████▏              | 151.1 MB 15.3 MB/s eta 0:00:09     |██████████████████▏             | 160.2 MB 3.7 MB/s eta 0:00:34     |██████████████████▎             | 160.6 MB 3.7 MB/s eta 0:00:34     |██████████████████▌             | 162.8 MB 3.7 MB/s eta 0:00:33     |███████████████████▎            | 169.7 MB 8.1 MB/s eta 0:00:14     |████████████████████▍           | 179.0 MB 12.8 MB/s eta 0:00:08     |███████████████████████▍        | 205.6 MB 13.6 MB/s eta 0:00:06     |████████████████████████▎

In [5]:
!apt-get update --fix-missing
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease                        
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]      
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]    
Get:5 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1512 kB]
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [991 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3231 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2286 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [957 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2799 kB]
Fetched 12.0 MB in 23s (533 kB/s)                                              
Reading package lists... Done
debconf: delaying package configuration, 

In [6]:
import pyspark.sql.types as T 
import pyspark.sql.functions as F 
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext

import pyspark.sql.functions as F
from pyspark.ml.fpm import FPGrowth

import findspark
findspark.init()
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


In [7]:
spark = SparkSession.builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/06 04:06:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
# conf = SparkConf().setAppName("base").setMaster("local[*]")
# spark = SparkContext(conf=conf).getOrCreate()
# spark.stop()

In [9]:
df = spark.read.options(header='true').csv('../data/raw/olist_order_items_dataset.csv')

df.show(2)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.90|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.90|        19.93|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
only showing top 2 rows



In [10]:
df = df.dropDuplicates(['order_id', 'product_id']).sort('order_id')

df.show(2)

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|00010242fe8c5a6d1...|            1|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-19 09:45:35| 58.90|        13.29|
|00018f77f2f0320c5...|            1|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-05-03 11:05:13|239.90|        19.93|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
only showing top 2 rows



In [11]:
df = df.groupBy('order_id').agg(F.collect_list('product_id')).sort('order_id')

df.show(2)

+--------------------+------------------------+
|            order_id|collect_list(product_id)|
+--------------------+------------------------+
|00010242fe8c5a6d1...|    [4244733e06e7ecb4...|
|00018f77f2f0320c5...|    [e5f2d52b802189ee...|
+--------------------+------------------------+
only showing top 2 rows



In [12]:
fpGrowth = FPGrowth(itemsCol='collect_list(product_id)', minSupport=0.00001, minConfidence=0.0001)

model = fpGrowth.fit(df)

In [13]:
model.freqItemsets.show()
items = model.freqItemsets.toPandas()

model.associationRules.show()
rules = model.associationRules.toPandas()

/root/miniconda3/envs/dsb/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
22/06/06 04:07:07 WARN DAGScheduler: Broadcasting large task binary with size 1495.3 KiB
22/06/06 04:07:08 WARN DAGScheduler: Broadcasting large task binary with size 1150.4 KiB
22/06/06 04:07:08 WARN DAGScheduler: Broadcasting large task binary with size 1149.8 KiB


+--------------------+----+
|               items|freq|
+--------------------+----+
|[ec58535d820b93ed...|   1|
|[be7f0912ba011267...|   5|
|[868766ed2172644f...|   1|
|[a94bae2ca6c0f653...|   3|
|[ceea7d6ebaeb5a36...|   1|
|[0bb1142b8037490f...|   1|
|[54df5de144ebb216...|   1|
|[75d3f426c010ad14...|   1|
|[3df78dbb35e3edc3...|   1|
|[f4470c119e77b798...|   1|
|[cb7433cfec8ba4a5...|   2|
|[71e5d0eb5a5fcc82...|   5|
|[ff4c1a248a5110d7...|   2|
|[0d0bc6cf9765ffe8...|   3|
|[ca4ac4e6aa504eee...|   1|
|[9c711b24a6ec15a6...|   1|
|[e449c89deefdb41b...|   1|
|[5e7cc48697a854bb...|   1|
|[55ba658d5c2f0b1f...|   1|
|[0e8c1dc0559f4a9a...|   1|
+--------------------+----+
only showing top 20 rows



/root/miniconda3/envs/dsb/lib/python3.9/site-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
22/06/06 04:07:09 WARN DAGScheduler: Broadcasting large task binary with size 1155.3 KiB
22/06/06 04:07:09 WARN DAGScheduler: Broadcasting large task binary with size 1155.3 KiB
22/06/06 04:07:10 WARN DAGScheduler: Broadcasting large task binary with size 1605.8 KiB
22/06/06 04:07:10 WARN DAGScheduler: Broadcasting large task binary with size 1604.5 KiB


+--------------------+--------------------+-------------------+------------------+--------------------+
|          antecedent|          consequent|         confidence|              lift|             support|
+--------------------+--------------------+-------------------+------------------+--------------------+
|[e65ab426efaf65e0...|[4d5bb93bfa70f67c...|                1.0|           49333.0|1.013520361624065...|
|[a5d89252158ff2fc...|[aaac7cb8bbb8fe12...|                1.0|           98666.0|1.013520361624065...|
|[50046ff3c7e88683...|[ac1fcaf7402bfbf0...|                1.0|           98666.0|1.013520361624065...|
|[50046ff3c7e88683...|[d5bf1e6386a450dd...|                1.0|           98666.0|1.013520361624065...|
|[50046ff3c7e88683...|[9eacfe990da87a66...|                1.0|           98666.0|1.013520361624065...|
|[ef7de0bbb0e0b011...|[7da9810448a8f92f...| 0.3333333333333333|16444.333333333332|1.013520361624065...|
|[5b8a5a9417210b1b...|[fe59a1e006df3ac4...|0.16666666666666666|1

In [14]:
items.to_csv('./mba_freq_items.csv')
rules.to_csv('./mba_pattern.csv')